In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
import pandas as pd
import math
import socket
import time

# TensorFlow

In [6]:
### Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Load Model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

### Load Video
video_path = '../../data/videos/'
sampleVideo = video_path + 'sampleVideo.mp4'
basicFour = video_path + 'basic_four.mp4'
basicTwo = video_path + 'basic_two.mp4'
swingDiff = video_path + 'swing_diff.mp4'
swingSame = video_path + 'swing_same.mp4'
swingRaw = video_path + 'swingRaw.mp4'
temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
temporalEasy = video_path + 'temporalEasyRaw.mp4'



### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


###
# Variables for drawing plot in real-time


if __name__ == "__main__":
    ### Variables
    numberOfPeople = 6
    lamdaVal = 0.885

    minBPD = 10.0
    maxBPD = 0.0
    
    saveNum = 0
    saveBPD = []

    ### Loading Video File
    cap = cv2.VideoCapture(swingSame)    ### Change the File Here!!
    while cap.isOpened():
        ret, frame = cap.read()

        ### Variables for each frame
        BPD = []
        
        # Resize image
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        input_img = tf.cast(img, dtype=tf.int32)
        
        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        keypoints_with_scores = keypoints_with_scores[:numberOfPeople]
        keypoints_only = np.delete(keypoints_with_scores,2,2)
        keypoints_only_body = np.delete(keypoints_only, [0,1,2,3,4], 1)
        
        print(keypoints_with_scores)



        # Render keypoints 
        loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
#         loop_through_people(frame, [keypoints_with_scores[0]], EDGES, 0.1)    # Check for first person.....

        
        cv2.imshow('Movenet Multipose', frame)
        
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

2023-04-24 23:32:39.695994: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


[[[3.6476201e-01 8.3545935e-01 7.4672717e-01]
  [3.5444689e-01 8.4398139e-01 6.6362476e-01]
  [3.5467491e-01 8.2751745e-01 7.0793146e-01]
  [3.7317765e-01 8.5271007e-01 8.1909257e-01]
  [3.7263161e-01 8.1701577e-01 8.6892736e-01]
  [4.3967950e-01 8.7313855e-01 9.3706298e-01]
  [4.4150072e-01 7.9451406e-01 9.1358018e-01]
  [4.2775244e-01 9.1751921e-01 5.4229635e-01]
  [4.3097040e-01 7.4447554e-01 6.7133361e-01]
  [4.1974068e-01 9.6330065e-01 7.1804374e-01]
  [4.1906297e-01 7.0499176e-01 8.6648190e-01]
  [6.1315268e-01 8.5410637e-01 9.4789529e-01]
  [6.1458999e-01 8.0876684e-01 9.7564673e-01]
  [7.5173706e-01 8.6477393e-01 8.5096806e-01]
  [7.4190927e-01 7.9379302e-01 9.3451542e-01]
  [8.6307871e-01 8.7179816e-01 8.7878174e-01]
  [8.6216605e-01 7.8050107e-01 8.4253448e-01]]

 [[3.6415362e-01 3.1265363e-01 6.0582286e-01]
  [3.5385332e-01 3.2019931e-01 6.7131323e-01]
  [3.5552782e-01 3.0514586e-01 6.7169404e-01]
  [3.7047368e-01 3.3021903e-01 7.8758365e-01]
  [3.7094384e-01 2.9671949e-01 8

: 

# TensorFlow Lite

In [6]:
### Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Load Model
interpreter  = tf.lite.Interpreter(model_path = '../../model/movenet/lite-model_movenet_multipose_lightning_tflite_float16_1.tflite')
interpreter.allocate_tensors() #have to do when working on tflite model 

### Load Video
video_path = '../../data/videos/'
sampleVideo = video_path + 'sampleVideo.mp4'
basicFour = video_path + 'basic_four.mp4'
basicTwo = video_path + 'basic_two.mp4'
swingDiff = video_path + 'swing_diff.mp4'
swingSame = video_path + 'swing_same.mp4'
swingRaw = video_path + 'swingRaw.mp4'
temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
temporalEasy = video_path + 'temporalEasyRaw.mp4'

### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


###
# Variables for drawing plot in real-time


if __name__ == "__main__":
    ### Variables
    numberOfPeople = 6
#     lamdaVal = 0.885

#     minBPD = 10.0
#     maxBPD = 0.0
    
#     saveNum = 0
    saveBPD = []

    ### Loading Video File
    cap = cv2.VideoCapture(temporalEasy)
    while cap.isOpened():
        ret, frame = cap.read()

        ### Variables for each frame
        BPD = []
        
        # Resize image
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        input_img = tf.cast(img, dtype=tf.int32)
        
        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        keypoints_with_scores = keypoints_with_scores[:numberOfPeople]
        keypoints_only = np.delete(keypoints_with_scores,2,2)
        keypoints_only_body = np.delete(keypoints_only, [0,1,2,3,4], 1)



        # Render keypoints 
        loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
        # loop_through_people(frame, [keypoints_with_scores[0]], EDGES, 0.1)    # Check for first person.....

        
        cv2.imshow('Movenet Multipose', frame)

        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()